# Mastering SQL Queries with BigQuery: Data Exploration and Analysis

#### Introduction

In this kernel, we embark on a journey of data exploration and analysis using the power of **BigQuery**, Google's cloud-based data warehouse. Our focus will be on **mastering SQL queries** to uncover valuable insights from complex datasets.

This project delves into the **"Catalonian Mobile Coverage"** dataset, a rich source of information on mobile network coverage, operator activity, and user behavior. The primary dataset, **mobile_data_2015_2017_cleaned**, has undergone thorough cleaning and normalization to ensure data quality.

#### Key Data Sources:


- **mobile_data_2015_2017_cleaned**: Our main dataset, providing insights into mobile network coverage, operator activity, and user behavior.

- **cat_percapita_income_by_province_2015_2017** and **cat_pop_by_province_2015_2017**: Additional datasets supplying data on per capita income and population by province.


#### Prominent SQL Commands and Functions:

   - **SQL SELECT Statements**: Querying data, filtering, sorting, and retrieving relevant information.
   - **Grouping and Aggregation**: Summarizing data with COUNT, SUM, AVG, and more.
   - **Window Functions**: Calculating rankings, tracking changes, and gaining insights.
   - **Conditional Statements**: Categorizing data using CASE statements.
   - **Subqueries**: Creating complex queries with embedded subqueries.
   - **Views**: Organizing and filtering data for in-depth analysis.
   - **Statistical Aggregate Functions**: Computing correlations, covariances, quartiles, and more.
   - **Date Functions**: Exploring date ranges, identifying top activity dates, and analyzing monthly and hourly patterns.
   - **Plotting and Visualization**: Presenting insights through interactive plots and visualizations.

#### Some Key Insights We Aim to Uncover:

- Identifying the top net_provider in Catalonia with the highest activity recorded.
- Finding the operator with the highest activity.
- Determining which net_provider/operator combination exhibits the highest average signal strength.
- Dividing operators into quartiles based on signal strength to assess performance variations.
- Analyzing changes in signal strength and net_provider speed within specific nets.
- Ranking net_provider and operators by record count and exploring the impact on rankings based on signal strength.


### Import libraries and modules

In [1]:
import pandas as pd
from google.cloud import bigquery
from IPython.display import display
from IPython.display import Markdown

import matplotlib.pyplot as plt
import seaborn as sns

### Import function: Interactive SQL Query to Pandas DataFrame Converter

Import the custom query_df and run_query functions from the 'query_functions.py' file to execute SQL queries using a pre-configured BigQuery client object.

In [2]:
from query_functions import query_df  # Execute the query and return the output as a DataFrame
from query_functions import run_query  # Execute the query without returning a DataFrame, used for INSERT, UPDATE, DELETE, etc.

### Datasets and Tables paths

In [3]:
# Catalonian mobile coverage (2015-2017)
mobile_data_cleaned = "bq-analyst-230590.project_cat_mobile_coverage_2015_2017.mobile_data_2015_2017_cleaned"

# Per capita income by catalan province (2015-2017)
percapita_income = "bq-analyst-230590.project_cat_mobile_coverage_2015_2017.cat_percapita_income_by_province_2015_2017"

# Catalan population and density by province (2015-2017)
pop_density = "bq-analyst-230590.project_cat_mobile_coverage_2015_2017.cat_pop_by_province_2015_2017"

### Preview datasets

Mobile Data Cleaned

In [4]:
# SQL query:
query = f"""
    SELECT *
    FROM `{mobile_data_cleaned}`
    LIMIT 100
    """

# Execute the query and store the result
mob_data = query_df(query)

# Show table
display(mob_data.head(),mob_data.tail())
print("Shape: \n", mob_data.shape)

,date,hour,lat,long,signal,net_provider,operator,description,net,speed,satellites,precision,position_provider,activity,postal_code,town_name,position_geom,province,year
0,2015-09-10,06:17:19,41.65941,2.45505,7,EE,EE,STATE_EMERGENCY_ONLY,3G,127.0,1.0,31.0,GPS,IN_VEHICLE,082943,Vallgorguina,POINT(2.45505 41.65941),Barcelona,2015
1,2015-09-08,23:14:38,41.67443,2.79337,17,EE,EE,STATE_EMERGENCY_ONLY,2G,24.5,2.0,8.0,GPS,IN_VEHICLE,170237,Blanes,POINT(2.79337 41.67443),Girona,2015
2,2015-09-09,17:54:37,42.13197,3.09622,11,EE,EE,STATE_EMERGENCY_ONLY,4G,64.9,6.0,25.0,GPS,IN_VEHICLE,170622,l'Escala,POINT(3.09622 42.13197),Girona,2015
3,2015-09-08,13:22:48,41.99098,2.79448,6,EE,EE,STATE_EMERGENCY_ONLY,2G,21.5,4.0,20.0,GPS,IN_VEHICLE,170792,Girona,POINT(2.79448 41.99098),Girona,2015
4,2015-09-09,16:03:41,41.95650,2.78532,9,EE,EE,STATE_EMERGENCY_ONLY,2G,144.2,10.0,15.0,GPS,IN_VEHICLE,172155,Vilablareix,POINT(2.78532 41.9565),Girona,2015


,date,hour,lat,long,signal,net_provider,operator,description,net,speed,satellites,precision,position_provider,activity,postal_code,town_name,position_geom,province,year
95,2015-12-29,11:34:26,41.40571,2.17606,25,Movistar,ONO,STATE_EMERGENCY_ONLY,3G,12.2,2.0,18.0,GPS,IN_VEHICLE,080193,Barcelona,POINT(2.17606 41.40571),Barcelona,2015
96,2015-03-20,19:32:01,41.37988,2.14658,14,Movistar,ONO,STATE_EMERGENCY_ONLY,3G,166.6,0.0,12.0,GPS,IN_VEHICLE,080193,Barcelona,POINT(2.14658 41.37988),Barcelona,2015
97,2015-12-29,12:54:41,41.38943,2.15378,10,Movistar,ONO,STATE_EMERGENCY_ONLY,3G,2.3,9.0,15.0,GPS,ON_FOOT,080193,Barcelona,POINT(2.15378 41.38943),Barcelona,2015
98,2015-05-27,16:59:12,41.40490,2.14568,14,Movistar,ONO,STATE_IN_SERVICE,3G,3.6,1.0,8.0,GPS,ON_FOOT,080193,Barcelona,POINT(2.14568 41.4049),Barcelona,2015
99,2015-05-07,13:21:45,41.40514,2.19603,26,Movistar,ONO,STATE_IN_SERVICE,3G,4.3,0.0,4.0,GPS,ON_FOOT,080193,Barcelona,POINT(2.19603 41.40514),Barcelona,2015


Shape: 
 (100, 19)


In [5]:
# List of columns
mob_data.columns

Index(['date', 'hour', 'lat', 'long', 'signal', 'net_provider', 'operator',
       'description', 'net', 'speed', 'satellites', 'precision',
       'position_provider', 'activity', 'postal_code', 'town_name',
       'position_geom', 'province', 'year'],
      dtype='object')

In [6]:
# Column types
mob_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               100 non-null    object 
 1   hour               100 non-null    object 
 2   lat                100 non-null    float64
 3   long               100 non-null    float64
 4   signal             100 non-null    int64  
 5   net_provider       100 non-null    object 
 6   operator           100 non-null    object 
 7   description        100 non-null    object 
 8   net                100 non-null    object 
 9   speed              100 non-null    float64
 10  satellites         100 non-null    float64
 11  precision          100 non-null    float64
 12  position_provider  100 non-null    object 
 13  activity           100 non-null    object 
 14  postal_code        100 non-null    object 
 15  town_name          100 non-null    object 
 16  position_geom      100 non-

As previously mentioned in the earlier kernel, during the retrieval of 'date' and 'hour' from Google BigQuery for the 'SQL Data Cleaning For Mobile Coverage Dataset', these fields were transformed to 'object' types. However, it's important to note that in the original Google BigQuery dataset, their formats are 'date' and 'time', respectively. If there is a need to update these formats in the original table, the required queries are provided below:

    (Please be aware that while we are displaying this data as a Python DataFrame for reference, the actual changes should be implemented in the original BigQuery dataset. We will directly query the original dataset, and we won't be utilizing this DataFrame for further analysis.)

In [7]:
# SQL query: in BigQuery,convert 'date' to date format and 'hour' to time format
query = f"""
    # Convert 'date' column to date data type
    UPDATE `{mobile_data_cleaned}`
    SET date = CAST(date AS DATE)
    """
# Execute the query
# run_query(query)


query = f"""
    # Convert 'date' column to date data type
    UPDATE `{mobile_data_cleaned}`
    SET hour = CAST(hour AS TIME)
    """

# Execute the query
# run_query(query)

In [8]:
# Summary Statistics of numerical features
mob_data.describe()

,lat,long,signal,speed,satellites,precision,year
count,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.0
mean,41.439965,2.214643,16.190000,20.72200,2.810000,18.040000,2015.0
std,0.128234,0.147043,5.868414,34.83661,2.725469,19.032094,0.0
min,41.307610,1.870990,3.000000,0.00000,0.000000,3.000000,2015.0
25%,41.398310,2.167782,13.000000,0.77500,0.000000,8.000000,2015.0
50%,41.404930,2.197315,14.000000,5.20000,2.000000,12.000000,2015.0
75%,41.420600,2.209245,20.250000,19.97500,4.000000,20.000000,2015.0
max,42.131970,3.096220,30.000000,166.60000,12.000000,112.000000,2015.0


In order to query more efficiently, even though we have a 'date' column in the 'mobile_data_cleaned' dataset (formatted as YYYY-MM-DD), we are going to **add a new column, 'year'**, which extracts the year from 'date'.

This simplification will make querying the 'mobile_data_cleaned' dataset more straightforward.

In [9]:
# SQL query:
query = f"""
    ALTER TABLE `{mobile_data_cleaned}`
    ADD COLUMN IF NOT EXISTS year INT64
    """
# Execute the query
run_query(query)

Query successfully executed, and the table has been updated.


In [10]:
# SQL query:
query = f"""
    UPDATE `{mobile_data_cleaned}`
    SET year = EXTRACT(YEAR FROM date)
    WHERE year IS NULL
    """
# Execute the query
run_query(query)

Query successfully executed, and the table has been updated.


In [11]:
# SQL query:
query = f"""
    SELECT
        DISTINCT year
    FROM `{mobile_data_cleaned}`
    """
# Execute the query
query_df(query)

,year
0,2017
1,2016
2,2015


Per Capita Income

In [12]:
# SQL query:
query = f"""
    SELECT *
    FROM `{percapita_income}`
    """

# Execute the query
query_df(query)

,year,province,per_capita_income
0,2015,Lleida,20136
1,2015,Tarragona,22486
2,2015,Barcelona,27214
3,2015,Girona,25200
4,2016,Tarragona,23130
5,2016,Lleida,20713
6,2016,Barcelona,27913
7,2016,Girona,25598
8,2017,Barcelona,28481
9,2017,Lleida,21091


Pop Density

In [13]:
# SQL query:
query = f"""
    SELECT *
    FROM `{pop_density}`
    LIMIT 100
    """
# Execute the query
query_df(query)

,province,year,population,sq_km,density_per_sq_km
0,Girona,2016,766273,5908,129.7
1,Girona,2017,766705,5908,129.8
2,Girona,2015,765783,5908,129.6
3,Lleida,2015,742138,12172,61.0
4,Lleida,2016,742099,12172,61.0
5,Lleida,2017,741884,12172,61.0
6,Barcelona,2015,5618162,7726,727.2
7,Barcelona,2016,5635085,7726,729.4
8,Barcelona,2017,5652301,7726,731.6
9,Tarragona,2015,811089,6303,128.7


### Execute queries:

**1. Filtering and Sorting**

**2. Aggregation**

**3. Join Operations**

**4. Left, Right, Inner and Full Join**

**5. Subqueries**

**6. Grouping and Aggregation**

**7. Data Validation**

**8. Case Statements**

**9. Statistical Aggregate Functions**

**10. Window Functions**

**11. Datetime Functions**

**12. Brief Exploratory Data Analysis**

#### 1. Filtering and Sorting

Retrieve all records from the '{mobile_2015_2017}' table for the year 2016 where the 'signal' strength is above a threshold of 100 and 'net' is not 'Undefined net'.

In [17]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT 
        year,
        operator,
        net,
        signal
    FROM `{mobile_data_cleaned}`
    WHERE 
        year = 2016
        AND signal > 50
        AND NET != 'Undefined net'
    ORDER BY signal DESC
    LIMIT 1000
    """

# Execute the query
query_df(query)  

,year,operator,net,signal
0,2016,Movistar,3G,99
1,2016,Movistar,2G,99
2,2016,Orange,3G,99
3,2016,Movistar,2G,99
4,2016,Movistar,2G,99
...,...,...,...,...
995,2016,Orange,4G,65
996,2016,Orange,4G,65
997,2016,Movistar,4G,65
998,2016,Vodafone,4G,65


Retrieve records from the '{mobile_2015_2017}' table for the year 2017 where 'net' is '4G' and 'description' is 'STATE_IN_SERVICE,' and for the year 2016 where 'net' is '3G' and 'signal' strength exceeds the year's average '3G' signal strength.

In [18]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      year,
      operator,
      net,
      description,
      signal,
      speed
    FROM `{mobile_data_cleaned}`
    WHERE
        # Condition 1: Select records for '4G' net in 2017 with STATE_IN_SERVICE description and speed > 100.0
      (year = 2017
      AND net = '4G'
      AND description = 'STATE_IN_SERVICE'
      AND speed > 100.0)
        # Condition 2: Select records for 3G network in 2016 with signal greater than the average signal strength for '3G' records in 2016
      OR
      (year = 2016
      AND net = '3G'
      AND signal > (
          # Subquery to calculate average signal strength for '3G' records in 2016
          SELECT
              AVG(signal)
          FROM `{mobile_data_cleaned}`
          WHERE
              year = 2016
              AND net = '3G')
       )
    ORDER BY 1 DESC
    LIMIT 1000;
    """
# Execute the query
query_df(query)    

,year,operator,net,description,signal,speed
0,2017,Pepephone,4G,STATE_IN_SERVICE,9,113.6
1,2017,Orange,4G,STATE_IN_SERVICE,3,101.6
2,2017,Movistar,4G,STATE_IN_SERVICE,8,112.8
3,2017,Orange,4G,STATE_IN_SERVICE,14,100.8
4,2017,Movistar,4G,STATE_IN_SERVICE,14,125.1
...,...,...,...,...,...,...
995,2016,Orange,3G,STATE_EMERGENCY_ONLY,22,3.3
996,2016,Orange,3G,STATE_EMERGENCY_ONLY,27,50.7
997,2016,Simyo,3G,STATE_EMERGENCY_ONLY,24,4.2
998,2016,Orange,3G,STATE_EMERGENCY_ONLY,16,5.0


Calculate and display the top 3 Orange 'operators' with the highest 'activity count' for the '4G' category, for both the years 2017 and 2016. Organize the results by the number of 'activity counts' in descending order and present the findings.

In [45]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
WITH OrangeOperators_4G_2017 AS ( 
    # Common Table Expression (CTE): Top-3 activity count '4G' Orange Operator in 2017
    SELECT
        year,
        net,
        operator,
        COUNT(*) activity_count
    FROM `{mobile_data_cleaned}`
    WHERE
      net_provider = 'Orange'
      AND net = '4G'
      AND year = 2017
    GROUP BY 1,2,3 
    ORDER BY 1 DESC, 2 DESC, 4 DESC
    LIMIT 3
    ),
    
    # CTE: Top-3 activity count '4G' Orange Operator in 2016
    OrangeOperators_4G_2016 AS ( 
    SELECT
        year,
        net,
        operator,
        COUNT(*) activity_count
    FROM `{mobile_data_cleaned}`
    WHERE
      net_provider = 'Orange'
      AND net = '4G'
      AND year = 2016
    GROUP BY 1,2,3 
    ORDER BY 1 DESC, 2 DESC, 4 DESC
    LIMIT 3
    )

SELECT
    year,
    net,
    operator,
    activity_count
FROM `OrangeOperators_4G_2017`
UNION ALL
SELECT
    year,
    net,
    operator,
    activity_count
FROM `OrangeOperators_4G_2016`
ORDER BY 1 DESC, 4 DESC
    """
# Execute the query
query_df(query) 

,year,net,operator,activity_count
0,2017,4G,Orange,173988
1,2017,4G,adamo,13295
2,2017,4G,Jazztel,11511
3,2016,4G,Orange,533603
4,2016,4G,Jazztel,108850
5,2016,4G,Simyo,15286


Retrieve the top 10 records from the '{mobile_2015_2017}' table for the year 2017 where the 'net' is '4G' and the 'description' is 'STATE_IN_SERVICE.' Additionally, for the year 2016, obtain the top 10 records where the 'net' is '3G,' and the 'signal' strength exceeds the average '3G' signal strength of the top 10% of '3G' records in terms of 'signal' strength in 2016. Sort the results by 'signal' strength in descending order.

#### 2. Aggregation

1. Calculate the average, maximum and standard deviation 'speed' of mobile data in the 'mobile 2015_2017' table for the year 2017.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT 
        ROUND(AVG(speed),2) avg_speed_2017,
        MAX(speed) max_speed_2017,
        ROUND(STDDEV_POP(speed),2) std_speed_2017
    FROM `{mobile_data_cleaned}`
    WHERE year = 2017
    """
# Execute the query
query_df(query)   

2. Calculate the average signal strength (signal) for each month in the year 2017 and order the results in descending order.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT 
        EXTRACT(MONTH FROM date) month,
        ROUND(AVG(signal),2) avg_signal_2017
    FROM `{mobile_data_cleaned}`
    WHERE year = 2017
    GROUP BY 1
    ORDER BY 2 DESC
    """

# Execute the query
query_df(query)  

3. Calculate the percentage of records where the activity is 'IN_VEHICLE' and 'net' is NOT 'Undefined net' in the '{mobile_2015_2017}' table for the year 2017.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
SELECT
    net,
    ROUND((in_vehicle_2017_count / total_count * 100), 2) AS in_vehicle_2017_perc
FROM
    
    # The first subquery (in_vehicle_subquery) calculates the count of records GROUPED BY 'net' where the activity is
    # 'IN_VEHICLE' and the 'net' is not 'Undefined net' for the year 2017.
     
    (SELECT 
        net,
        COUNT(*) AS in_vehicle_2017_count
    FROM `{mobile_data_cleaned}`
    WHERE year = 2017
        AND activity = 'IN_VEHICLE'
        AND net != 'Undefined net'
    GROUP BY 1) in_vehicle_subquery,
    
    # The second subquery (total_subquery) calculates the total count of records where the activity is 'IN_VEHICLE' 
    # and the 'net' is not 'Undefined net' for the year 2017.
    
    (SELECT COUNT(*) AS total_count
     FROM `{mobile_data_cleaned}`
     WHERE year = 2017
        AND activity = 'IN_VEHICLE'
        AND net != 'Undefined net') total_subquery;

    """
# Execute the query
query_df(query)   

#### 3. Join Operations

1. Join the '{mobile_2015_2017}' table with the '{pop_density}' table to retrieve records from both tables for the year 2016 and calculate the average signal strength (signal) for each 'province'.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT 
      m.province,
      p.sq_km,
      p.population,
      p.density_per_sq_km,
      ROUND(AVG(m.signal),2) avg_signal
    FROM `{mobile_data_cleaned}` m
    JOIN `{pop_density}` p 
      ON m.province = p.province AND m.year = p.year
    WHERE m.year = 2016
    GROUP BY m.province, p.sq_km, p.population, p.density_per_sq_km
    ORDER BY p.density_per_sq_km DESC;
    """
# Execute the query
query_df(query) 

2. Join the '{mobile_2015_2017}' table with the '{percapita_income}' table to find the province with the best 4G network coverage (highest number of records with '4G' in the 'net' column) in the year 2015 and retrieve records of its 'per_capita_income'.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT 
      m.province,
      m.net,
      COUNT(m.net) records_count_2015,
      c.per_capita_income AS per_capita_income_2015
    FROM `{mobile_data_cleaned}` m
    JOIN `{percapita_income}` c
      ON m.year = c.year AND m.province = c.province
    WHERE m.net = '4G' AND m.year = 2015
    GROUP BY 1,2,4
    ORDER BY COUNT(*) DESC
    LIMIT 1
    """
# Execute the query
query_df(query) 

3. Find the province with the highest average signal strength (signal) in the year 2017. Then, join this result with the "{percapita_income}" table to get the per capita income for that province in 2017.


In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT 
      m.province,
      ROUND(AVG(m.signal),2) avg_signal_2017,
      C.per_capita_income
    FROM `{mobile_data_cleaned}` m
    JOIN `{percapita_income}` c
      ON m.province = c.province AND m.year = c.year
    WHERE m.year = 2017
    GROUP BY m.province, c.per_capita_income
    ORDER BY avg_signal_2017 DESC
    LIMIT 1;
    """
# Execute the query
query_df(query)  

4. Calculate the correlation coefficient between the average signal strength (signal) and population density for all provinces.


In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
WITH avg_signal_table AS (
    SELECT
        m.year,
        m.province,
        AVG(m.signal) avg_signal
    FROM
        `{mobile_data_cleaned}` m
    GROUP BY m.year, m.province
)

SELECT 
    # Calculate Pearson Correlation
    CORR(t.avg_signal, p.density_per_sq_km) corr_signal_density
FROM avg_signal_table t
JOIN `{pop_density}` p 
  ON t.province = p.province AND t.year = p.year
;
"""
# Execute the query
query_df(query) 

    -  This value suggests a moderately strong positive linear correlation between the average signal strength (avg_signal) and population density (density_per_sq_km) for the given provinces and years in the dataset. A correlation coefficient of 0.71 indicates that as population density increases, the average signal strength tends to increase as well.

5. Calculate the correlation coefficient between the average signal strength (signal) and per capita income for all provinces in the year 2017.


In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
WITH avg_signal_table AS (
    SELECT
        m.year,
        m.province,
        AVG(m.signal) avg_signal
    FROM
        `{mobile_data_cleaned}` m
    GROUP BY m.year, m.province
)

SELECT 
    CORR(t.avg_signal, c.per_capita_income) corr_signal_income
FROM avg_signal_table t
JOIN `{percapita_income}` c 
  ON t.province = c.province AND t.year = c.year
;
"""
# Execute the query
query_df(query)  

    - This value suggests a positive linear correlation too, but it is weaker than the correlation between average signal strength (avg_signal) and population density (density_per_sq_km) that was calculated in the previous query.

6. Group the data in the '{mobile_2015_2017}' table by net type (e.g., 2G, 3G, 4G) and calculate the average signal strength for each type. Join this data with the "{percapita_income}" table to compare the per capita income by network type in the year 2017.


In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
WITH income_2017 AS (
    SELECT
        c.year,
        c.province,
        c.per_capita_income
    FROM
        `{percapita_income}` c
    WHERE
        c.year = 2017
)

SELECT
    m.net,
    ROUND(AVG(m.signal), 2) avg_signal,
    ROUND(AVG(t.per_capita_income),2) avg_percapita_income
FROM
    `income_2017` t
JOIN `{mobile_data_cleaned}` m 
ON t.province = m.province AND t.year = m.year
GROUP BY 1
ORDER BY 2 DESC;
"""
# Execute the query
query_df(query)

7. Calculate and compare the annual growth in average signal strength ('signal'), population, and per capita income for each province by joining the '{mobile_data_cleaned},' '{pop_density},' and '{percapita_income}' datasets.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
# Create a Common Table Expression (CTE) to retrieve the data for each year and province
WITH annual_data AS (
    SELECT
        m.year,
        m.province,
        ROUND(AVG(m.signal), 2) avg_signal,
        COUNT(*) activity_count,
        p.population,
        c.per_capita_income
    FROM `{mobile_data_cleaned}` m
    JOIN `{pop_density}` p
        ON m.province = p.province AND m.year = p.year
    JOIN `{percapita_income}` c
        ON m.province = c.province AND m.year = c.year
    GROUP BY 1, 2, 5, 6
    ORDER BY 2 ASC, 1 DESC
)

# Calculate annual growth percentages for each feature and province
SELECT
    a1.year,
    a1.province,
    a1.avg_signal,
    # Signal Growth
    ROUND((a1.avg_signal - a2.avg_signal) / a2.avg_signal * 100, 2) AS avg_signal_growth,
    # Activity Count Growth
    ROUND((a1.activity_count - a2.activity_count) / a2.activity_count * 100, 2) AS activity_count_growth,
    # Population Growth
    ROUND((a1.population - a2.population) / a2.population * 100, 2) AS population_growth,
    # Per Capita Income Growth
    ROUND((a1.per_capita_income - a2.per_capita_income) / a2.per_capita_income * 100, 2) AS per_capita_income_growth
FROM annual_data a1
# Join the 'annual_data' a2 to compare data for the same province and the next year (a2.year +1)
JOIN annual_data a2
    ON a1.province = a2.province -- Match data for the same province
    AND a1.year = a2.year + 1 -- Compare data for the current year with data from the next year
ORDER BY a1.province ASC, a1.year DESC
"""
# Execute the query
query_df(query)

    - Check and compare annual activity_count (focus 2017)

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
SELECT
    year,
    COUNT(*) activity_count
FROM `{mobile_data_cleaned}`
GROUP BY 1
ORDER BY 1 DESC
"""
# Execute the query
query_df(query)

Activity counts for the year 2017 show significantly fewer records compared to 2016 and 2015, which explains the negative growth observed in the previous query.

#### 4. Left, Right, Inner and Full join:

We will now introduce simulated missing data into the 'pop_density' table by adding rows with NULL values in some of its columns. This step allows us to observe the effects of different types of joins. Subsequently, we will perform a series of join operations with the 'mobile_2015_2017' table.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# pop_density columns: province, year, population, sq_km, density_per_sq_km

query = f"""
    INSERT INTO `{pop_density}`(year, population, sq_km)
    VALUES 
        (2015, 600000, 500),
        (2014, 530000, 490),
        (2016, 420000, 400),
        (2017, 385000, 380);
    
    INSERT INTO `{pop_density}`(province, year, sq_km)
    VALUES
        ('Costa Brava', 2017, 678),
        ('Pirineus', 2015, 10000);
    """
# Execute the query
run_query(query)  

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# pop_density columns: province, year, population, sq_km, density_per_sq_km

query = f"""   
    INSERT INTO `{pop_density}`(province, year, sq_km)
    VALUES
        ('Costa Brava', 2017, 678),
        ('Pirineus', 2015, 10000);
    """
# Execute the query
run_query(query)  

    - Output:

In [ ]:
# Check updated pop_density dataset

query = f"""
    SELECT *
    FROM `{pop_density}`;
    
    """
# Execute the query
query_df(query)  

    - a. LEFT JOIN: Count the number of records for each province and year, including provinces with NULL values in the pop_density table.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      d.province,
      d.year,
      COUNT(m.signal) activity_count
    FROM `{pop_density}` d
    LEFT JOIN `{mobile_data_cleaned}` m
        ON d.province = m.province
        AND d.year = m.year
    GROUP BY 1,2
    ORDER BY 1,2 DESC;
    """
# Execute the query
query_df(query)  

Left Join: include all rows from {pop_density} with NULL values in the columns from {mobile_2015_2017}

    - b. RIGHT JOIN: Count the number of records for each province and year, excluding provinces with NULL values in the pop_density table.


In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      d.province,
      d.year,
      ROUND(AVG(m.signal),2) avg_signal,
      ROUND(MAX(m.speed),2) max_speed
    FROM `{pop_density}` d
    RIGHT JOIN `{mobile_data_cleaned}` m
        ON d.province = m.province
        AND d.year = m.year
    GROUP BY 1,2
    ORDER BY 1,2 DESC;
    """
# Execute the query
query_df(query) 

Right Join: includes all rows from {mobile_2015_2017} and fill in NULL values from {pop_density} where there are no matches.

    - c. INNER JOIN: Count the number of records for each province and year, including provinces from the pop_density table and any additional data from the mobile_2015_2017 table.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      d.province,
      d.year,
      ROUND(AVG(m.signal),2) avg_signal,
      ROUND(MAX(m.speed),2) max_speed
    FROM `{pop_density}` d
    INNER JOIN `{mobile_data_cleaned}` m
        ON d.province = m.province
        AND d.year = m.year
    GROUP BY 1,2
    ORDER BY 1,2 DESC;
    """
# Execute the query
query_df(query)  

Inner Join: includes only records with matching province values in both tables and exclude rows with NULL values from {pop_density}.

    - d. FULL JOIN: Count the number of records for each province and year, including all available data from both tables.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      d.province,
      d.year,
      ROUND(AVG(m.signal),2) avg_signal,
      ROUND(MAX(m.speed),2) max_speed
    FROM `{pop_density}` d
    FULL JOIN `{mobile_data_cleaned}` m
        ON d.province = m.province
        AND d.year = m.year
    GROUP BY 1,2
    ORDER BY 1,2 DESC;
    """
# Execute the query
query_df(query)    

Full join: includes all rows from both tables and provide NULL values for {mobile_2015_2017} where there are no matches.

    - Delete added rows

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# Delete all new rows
query = f"""
    DELETE FROM `{pop_density}`
    WHERE 
        province IS NULL 
        OR province IN ('Costa Brava', 'Pirineus');
    """
# Execute the query
run_query(query)    

#### 4. Subqueries

Write a SQL subquery to find the provinces in the 'pop density' table where the population in 2016 is higher than the average population for all provinces.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      province,
      population
    FROM `{pop_density}`
    WHERE
      year = 2016
      AND population > (
        SELECT AVG(population)
        FROM `{pop_density}`
      );
    """
# Execute the query
query_df(query)

#### 5. Grouping and Aggregation

Group the 'mobile 2015_2017' table by 'network' type and calculate the average 'signal' strength and the maximum 'speed' for each network type in the year 2017.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      net,
      ROUND(AVG(signal),2) avg_signal,
      MAX(speed) max_speed
    FROM `{mobile_data_cleaned}`
    WHERE
      year = 2017
    GROUP BY 1;
    """
# Execute the query
query_df(query) 

#### 7. Data Validation

1. Identify and list all records in the '{mobile_2015_2017}' table with missing values in the 'position_provider' column.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
     *
    FROM `{mobile_data_cleaned}`
    WHERE
      position_provider IS NULL
    LIMIT 1000;
    """
# Execute the query
query_df(query)

#### 8. Case Statements

1. Update the 'signal' column in the '{mobile_2015_2017}' table, setting it to 'Above avg' when the 'signal' is above 14, 'On Avg' when between 13 and 14, and 'Under Avg' when below 13.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      max(signal) max_signal,
      min(signal) min_signal,
      avg(signal) avg_signal
    FROM `{mobile_data_cleaned}`
    """
# Execute the query
query_df(query)   

    - Create a new column to store the updated value

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# Create a new column to store the updated value
query = f"""
    ALTER TABLE `{mobile_data_cleaned}`
    ADD COLUMN signal_eval STRING;
    """

# Execute the query
run_query(query)    

    - Update values

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# Update values
query = f"""
    UPDATE `{mobile_data_cleaned}`
    SET signal_eval =
        CASE
            WHEN signal < 13 THEN 'Under Avg'
            WHEN signal BETWEEN 13 AND 14 THEN 'On Avg'
            WHEN signal > 14 THEN 'Above Avg'
            END
        WHERE signal_eval IS NULL;
    """

# Execute the query and store the result in the DataFrame
run_query(query)    

    - Check output

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# Check results
query = f"""
    SELECT
      signal,
      signal_eval
    FROM `{mobile_data_cleaned}`
    LIMIT 100
    """
# Execute the query
query_df(query) 

    - Delete 'signal_eval' column

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# Delete the added column
query = f"""
    ALTER TABLE `{mobile_data_cleaned}`
    DROP COLUMN signal_eval;
    """

# Execute the query
run_query(query)    

#### 9. Statistical Aggregate Functions

1. Calculate the Pearson correlation coefficient between different features pairs in the '{mobile_2015_2017}' table for all years in the dataset.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      CORR(signal, speed) AS signal_speed,
      CORR(signal, hour_24h) AS signal_hour,
      CORR(signal, CAST(postal_code AS INT64)) AS signal_postalcode,
      CORR(lat, long) AS lat_long,
      CORR(precission, satellites) AS precision_satellites,
      CORR(precission, speed) AS precision_speed,
      CORR(precission, signal) AS precision_signal,
      CORR(precission, CAST(postal_code AS INT64)) AS precission_postalcode,
      CORR(satellites, CAST(postal_code AS INT64)) AS satellites_postalcode
    FROM `{mobile_data_cleaned}`
    """
# Execute the query
query_df(query)  

2. Calculate the covariance between 'signal' strength and 'speed' in the '{mobile_2015_2017}' table.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      COVAR_POP(signal, speed) AS COVAR_signal_speed,
      CORR(signal, speed) AS CORR_signal_speed
    FROM `{mobile_data_cleaned}`
    """
# Execute the query
query_df(query)    

    - Covariance measures the degree to which two variables change together. In this case, the negative value of -0.95 suggests that there is a negative linear relationship between "signal" and "speed." When one variable increases, the other tends to decrease, and vice versa. However, the correlation coefficient (that unlike covariance, the correlation coefficient is a standardized measure within the range of -1 to 1.), which is approximately -0.004, indicates an extremely weak linear relationship, close to zero, suggesting that there is no substantial linear correlation between these two variables. In essence, while there is a negative covariance, the correlation coefficient underscores the absence of a strong linear connection between "signal" and "speed" in the dataset.

3. Calculate the standard deviation of 'signal' strength in the '{mobile_2015_2017}' table for the year 2017.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      ROUND(STDDEV_POP(signal),2) AS STD_signal,
      ROUND(MIN(signal),2) AS MIN_signal,
      ROUND(MAX(signal),2) AS MAX_signal,
      ROUND(AVG(signal),2) AS AVG_signal      
    FROM `{mobile_data_cleaned}`
    WHERE year = 2017
    """
# Execute the query
query_df(query)

    - These statistics provide insights into the variability and range of signal strengths in the dataset for the year 2017. The standard deviation of 7.18 indicates that signal strengths vary from the average by an average of approximately 7.18 units, and the signal strength ranged from 0.0 to 65.0 during that year.

4. Calculate the variance of 'speed' in the '{mobile_2015_2017}' table for the year 2017.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      VAR_POP(speed) VAR_speed,
      STDDEV_POP(speed) AS STD_speed,
      MAX(speed) - MIN(speed) AS Range_speed
    FROM `{mobile_data_cleaned}`
    WHERE year=2017
    """
# Execute the query
query_df(query)  

    - The statistics for the 'speed' variable in the year 2017 reveal a substantial degree of variability, as evidenced by the high population variance, standard deviation, and range. This variation suggests that the data points for 'speed' exhibit a wide dispersion from the mean and cover a significant range, reflecting diverse speed measurements throughout the year 2017.

#### 10. Window Functions

Before we start working with window functions, we will set the stage by inserting sample data into the {mobile_data_cleaned} table. This data will help us explore the distinctions between the RANK() and DENSE_RANK() window functions when ranking operators within networks.

Additionally, we will create a **view**, {netwop_4G_view}, to filter and extract a specific subset of the data, focusing on '4G' networks. This view will serve as the basis for our exploration of window functions, allowing us to rank operators and gain insights into their network activity.

    - Insert new rows

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    INSERT INTO `{mobile_data_cleaned}` (network, operator, net, signal, speed)
    VALUES
        ('invented_network', 'invented_operator', '4G', 11, 13),
        ('invented_network', 'invented_operator', '4G', 12, 14),
        ('invented_network', 'invented_operator2', '4G', 12, 14)
    """

# Execute the query
run_query(query)  

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
        *
    FROM `{mobile_data_cleaned}`
    WHERE REGEXP_CONTAINS(operator,'invented')
    """

# Execute the query
query_df(query)  

    - Create View

In [ ]:
# Network & Operator (unique pairs) and 4G net table (2015_2017) path 
netwop_4G_view = "bq-analyst-230590.project_cat_mobile_coverage_2015_2017.netw_op_4G_view"

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
    CREATE OR REPLACE VIEW `{netwop_4G_view}` AS
    SELECT
      network,
      operator,
      COUNT(*) AS activity_count,
      ROUND(AVG(signal), 2) AS avg_signal,
      ROUND(AVG(speed), 2) AS avg_speed
    FROM `{mobile_data_cleaned}`
    WHERE net = '4G'
    GROUP BY network, operator
    ORDER BY network
    """

# Execute the query
run_query(query)  

    - Output

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
    SELECT
      *
    FROM `{netwop_4G_view}`
    """

# Execute the query
query_df(query)  

1. How do networks within the '4G' network rank based on the number of operators, and which networks are at the top 10 in terms of operator count?

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

# Rank Vs Dense Rank

query = f"""
SELECT
  network,
  COUNT(operator) operator_count,
  RANK() OVER(ORDER BY COUNT(operator) DESC) overall_rank_num_op,
  DENSE_RANK() OVER(ORDER BY COUNT(operator) DESC) overall_DENSE_rank_num_op
FROM `{netwop_4G_view}`
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
    """

# Execute the query
query_df(query)  

    - Drop 'invented' rows from original dataset

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
    DELETE FROM `{mobile_data_cleaned}`
    WHERE REGEXP_CONTAINS(operator,'invented')
    """

# Execute the query
run_query(query)  

2. Within '4G' networks with multiple operators, how are operators ranked based on their activity count, and what is the comparative distribution of their rankings within their respective networks?

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
WITH FilteredNetworks AS (
  SELECT
    network
  FROM `{netwop_4G_view}`
  GROUP BY network
  HAVING COUNT(DISTINCT operator) > 1
)

SELECT
  network,
  operator,
  activity_count,
  RANK() OVER (PARTITION BY network ORDER BY activity_count) AS rank_within_network,
  PERCENT_RANK() OVER (PARTITION BY network ORDER BY activity_count) AS percent_rank,
  DENSE_RANK() OVER (PARTITION BY network ORDER BY activity_count) AS dense_rank
FROM `{netwop_4G_view}`
WHERE network IN (SELECT network FROM FilteredNetworks)
    """

# Execute the query
query_df(query)  

3. How does the moving average signal strength within the 'Orange' network for each operator compare to their respective average signal strength?

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
SELECT
  network,
  operator,
  avg_signal,
  AVG(avg_signal) OVER (PARTITION BY network ORDER BY avg_signal ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING) AS moving_avg_signal
FROM `{netwop_4G_view}`
WHERE network = 'Orange'
GROUP BY 1,2,3
ORDER BY 1, 2 ASC
    """

# Execute the query
query_df(query)

    - To calculate the moving average, it considers the signal strength values of each operator and computes the average of the surrounding values within a window of two preceding and two following rows (including the operator's value). This provides an estimate of signal strength variations for each operator within the network, which can be useful for identifying trends and deviations in signal strength over time.

4. Can we divide operators in the '4G' network into quartiles based on their average signal strength, and what insights can we gain into the performance variation within each quartile?

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
WITH Quartiles AS (
    SELECT
        operator,
        avg_signal,
        # Divide operators into quartiles (1-4), with 1 being the lowest and 4 being the highest signal strength quartile.
        NTILE(4) OVER (PARTITION BY network ORDER BY avg_signal) AS signal_quartile 
    FROM `{netwop_4G_view}`
    )

SELECT
  signal_quartile,
  COUNT(operator) AS operator_count,
  MIN(avg_signal) AS min_signal,
  MAX(avg_signal) AS max_signal,
  AVG(avg_signal) AS avg_signal
FROM Quartiles
GROUP BY signal_quartile
ORDER BY signal_quartile
    """

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
WITH Quartiles AS (
    SELECT
        operator,
        avg_signal,
        # Divide operators into quartiles (1-4), with 1 being the lowest and 4 being the highest signal strength quartile.
        NTILE(4) OVER (PARTITION BY network ORDER BY avg_signal) AS signal_quartile 
    FROM `{netwop_4G_view}`
    )

SELECT
  signal_quartile,
  COUNT(operator) AS operator_count,
  MIN(avg_signal) AS min_signal,
  MAX(avg_signal) AS max_signal,
  AVG(avg_signal) AS avg_signal
FROM Quartiles
GROUP BY signal_quartile
ORDER BY signal_quartile
    """

# Execute the query
query_df(query)

    - This analysis can be valuable for assessing the performance variation among different telecommunication providers (networks). It helps in understanding how the operators within each provider compare in terms of signal strength. For example, Quartile 4 includes operators with the highest signal strength, while Quartile 1 includes operators with the lowest signal strength. Operators in Quartiles 2 and 3 have signal strengths that fall in between.

    - This information can be useful for network administrators and planners who are interested in evaluating signal strength across various telecommunication providers, which enables them to make informed decisions regarding network optimization and performance improvements.

5. What insights can we gain about signal strength changes and network speed variations among operators in the '4G' network, especially within the 'Orange' network category, using LEAD and LAG window functions without considering time-based changes?

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
SELECT
    network,
    operator,
    avg_signal,
    LEAD(avg_signal) OVER (PARTITION BY network ORDER BY avg_signal) lead_avg_signal
    FROM `{netwop_4G_view}`
WHERE network = 'Orange'
GROUP BY 1,2,3
ORDER BY 3 ASC
    """

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
SELECT
    network,
    operator,
    avg_speed,
    # LAG( ,3) takes the avg_speed of row_num - 3.
    LAG(avg_speed,3) OVER(PARTITION BY network ORDER BY avg_speed) lag_avg_speed
FROM `{netwop_4G_view}`
WHERE network = 'Orange'
GROUP BY 1,2,3
ORDER BY 3 ASC
    """

# Execute the query
query_df(query)

    - 6. AAmong 'Orange' operators in the '4G' network, how does their ranking based on signal strength compare to their ranking based on the number of records? Do 'Orange' operators with strong signal strength also have a larger volume of records, and does this impact their overall ranking?

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
SELECT
    operator,
    avg_signal AS signal_strength,
    ROW_NUMBER() OVER (PARTITION BY network ORDER BY avg_signal DESC) AS signal_rank,
    activity_count AS record_count,
    ROW_NUMBER() OVER (PARTITION BY network ORDER BY activity_count DESC) AS record_rank
FROM `{netwop_4G_view}`
WHERE network = 'Orange'
ORDER BY signal_rank;
    """

# Execute the query
query_df(query)

    - We can observe that the ranking based on signal strength doesn't necessarily correlate with the ranking based on the number of records. Operators like RACC, with the third-highest signal strength, are ranked seventh (out of nine) in terms of record count, indicating a low presence in the network.

### 11. Dates

Date range

In [ ]:
# SQL query
query = f"""
    SELECT
        MIN(date) AS first_date_recorded,
        MAX(date) AS last_date_recorded,
        DATE_DIFF(MAX(date), MIN(date), DAY) AS total_days_recorded
    FROM `{mobile_data_cleaned}`
    """
# Execute the query
query_df(query) 

Top 10 dates with the highest activity

In [ ]:
# SQL query
query = f"""
    SELECT
      date,
      COUNT(*) AS record_count
    FROM `{mobile_data_cleaned}`
    GROUP BY date
    ORDER BY record_count DESC
    LIMIT 10
    """
# Execute the query
query_df(query) 

Monthly Activity Rank within Quarters and Across the Year

In [ ]:
# SQL query
query = f"""
    WITH MonthlyCounts AS (
        SELECT
            EXTRACT(QUARTER FROM date) AS quarter,
            EXTRACT(MONTH FROM date) AS month,
            COUNT(*) AS record_count
        FROM `{mobile_data_cleaned}`
        GROUP BY quarter, month
    )

    SELECT
      quarter,
      month,
      record_count,
      RANK() OVER (ORDER BY record_count DESC) AS month_rank
    FROM MonthlyCounts
    ORDER BY quarter, record_count DESC;
    """
# Execute the query and store the result in the DataFrame
month_rank_df = query_df(query) 

month_rank_df

In [ ]:
# Create a custom color palette for each month within a quarter
palette = sns.color_palette("tab10", n_colors=12)  # Use n_colors=12 for 12 months

# Pivot the DataFrame to have months as columns for stacking
stacked_df = month_rank_df.pivot(index='quarter', columns='month', values='record_count')

# Bar plot with stacked bars for each quarter and different colors for months
plt.figure(figsize=(12, 6))
ax = stacked_df.plot(kind='bar', stacked=True, color=palette)
plt.title('Stacked Bar Plot of Quarter-wise Record Count')
plt.xlabel('Quarter')
plt.ylabel('Record Count')
plt.legend(title='Month', loc='upper right', bbox_to_anchor=(1.15, 1))
plt.ticklabel_format(style='plain', axis='y')  # Disable scientific notation on y-axis
plt.show()


In [ ]:
# Define custom color palettes for each quarter
colors_first_quarter = sns.color_palette("Blues", n_colors=3)  # Blue colors for months 1, 2, 3
colors_second_quarter = sns.color_palette("Greens", n_colors=3)  # Green colors for months 4, 5, 6
colors_third_quarter = sns.color_palette("Oranges", n_colors=3)  # Orange colors for months 7, 8, 9
colors_fourth_quarter = sns.color_palette("Purples", n_colors=3)  # Purple colors for months 10, 11, 12

# Pivot the DataFrame to have months as columns for stacking
stacked_df = month_rank_df.pivot(index='quarter', columns='month', values='record_count')

# Create a stacked bar plot with distinct colors for each month within a quarter
plt.figure(figsize=(12, 6))

# Loop through each quarter and plot stacked bars with the custom color palette
quarters = stacked_df.index
for quarter, color_palette in zip(quarters, [colors_first_quarter, colors_second_quarter, colors_third_quarter, colors_fourth_quarter]):
    quarter_data = stacked_df.loc[quarter]
    quarter_data.plot(kind='bar', stacked=True, color=color_palette, label=f'Quarter {quarter}')

# Calculate the overall average for all values
overall_average = stacked_df.stack().mean()

# Plot a red horizontal line for the overall average
plt.axhline(y=overall_average, color='red', linestyle='--', label=f'Avg. Overall')

plt.title('Stacked Bar Plot of Quarter-wise Record Count with Distinct Colors for Months')
plt.xlabel('Month')
plt.ylabel('Record Count')
plt.legend(title='Legend', loc='upper right', bbox_to_anchor=(1.15, 1))
plt.ticklabel_format(style='plain', axis='y')  # Disable scientific notation on y-axis
plt.show()


### Hours

Hourly Period Counts

In [ ]:
# SQL query
query = f"""
    SELECT
      CASE
        WHEN EXTRACT(HOUR FROM hour) BETWEEN 0 AND 6 THEN 'Dawn'
        WHEN EXTRACT(HOUR FROM hour) BETWEEN 7 AND 12 THEN 'Morning'
        WHEN EXTRACT(HOUR FROM hour) BETWEEN 13 AND 18 THEN 'Afternoon'
        ELSE 'Night'
      END AS period,
      COUNT(*) AS record_count
    FROM `{mobile_data_cleaned}`
    GROUP BY 1
    ORDER BY 2 DESC
    """

# Execute the query and store the result in the DataFrame
hourly_period_counts = query_df(query)

hourly_period_counts

In [ ]:
import plotly.express as px

# Define the custom order and colors
custom_order = ["Dawn", "Morning", "Afternoon", "Night"]

# Create an interactive pie chart with Plotly
fig = px.pie(hourly_period_counts, values='record_count', names='period', 
             title='Hourly Period Counts', 
             hover_data=['record_count'], 
             labels={'record_count': 'Record Count'},
             category_orders={"period": custom_order})

# Customize the layout (optional)
fig.update_traces(textinfo='percent+label', pull=[0.1, 0.1, 0.1, 0.1])

# Show the chart
fig.show()

# Save the plot as an HTML file in the 'Python Plots' folder
plot_file_path = "Python Plots/hourly_period_counts.html"
fig.write_html(plot_file_path)

[View the Plot](./Python%20Plots/hourly_period_counts.html)

### Towns and Province activity

Top 50 Towns with the Highest Mobile Activity Recorded

In [ ]:
# SQL query
query = f"""
    SELECT
      postal_code,
      town_name,
      COUNT(*) AS record_count
    FROM
      `{mobile_data_cleaned}`
    WHERE postal_code IS NOT NULL
    GROUP BY postal_code, town_name
    ORDER BY 3 DESC
    LIMIT 50
    """

# Execute the query
query_df(query)

In [ ]:
# SQL query
query = f"""
WITH ProvinceActivity AS (
  SELECT
      CASE
        WHEN LEFT(CAST(postal_code AS STRING), 2) = '08' THEN 'Barcelona'
        WHEN LEFT(CAST(postal_code AS STRING), 2) = '25' THEN 'Lleida'
        WHEN LEFT(CAST(postal_code AS STRING), 2) = '17' THEN 'Girona'
        WHEN LEFT(CAST(postal_code AS STRING), 2) = '43' THEN 'Tarragona'
        ELSE 'Not defined'
      END AS province,
      COUNT(*) AS record_count
  FROM `{mobile_data_cleaned}`
  GROUP BY province
)

SELECT
    province,
    record_count,
    ROUND((record_count / SUM(record_count) OVER ()) * 100,2) AS percentage
FROM ProvinceActivity
ORDER BY 2 DESC;
    """
# Execute the query
query_df(query)

### Network, operators and signal

In [ ]:
# SQL query
query = f"""
SELECT
  network,
  operator,
  COUNT(*) record_count,
  ROUND(AVG(signal),1) avg_netw_signal
FROM
  `{mobile_data_cleaned}`
GROUP BY 1,2
ORDER BY 3 DESC;
    """

# Execute the query and store the result in the DataFrame
netw_oper_signal = query_df(query)

netw_oper_signal

In [ ]:
networks = netw_oper_signal['network'].nunique()
operators = netw_oper_signal['operator'].nunique()

display(Markdown(f"There are {networks} unique networks and {operators} distinct operators"))

Which (national) network has the highest activity recorded?

In [ ]:
# Group the DataFrame by the "network" column and calculate the total activity
network_activity = netw_oper_signal.groupby('network')['record_count'].sum().reset_index()

# Find the network with the highest total activity
max_activity_network = network_activity[network_activity['record_count'] == network_activity['record_count'].max()]

# Print the network with the highest total activity
max_activity_network

And the operator?

In [ ]:
# Group the DataFrame by the "operator" column and calculate the total activity
operator_activity = netw_oper_signal.groupby('operator')['record_count'].sum().reset_index()

# Find the operator with the highest total activity
max_activity_operator = operator_activity[operator_activity['record_count'] == operator_activity['record_count'].max()]

# Print the operator with the highest total activity
max_activity_operator

Which network/operator has the highest average signal?

In [ ]:
# Find the index of the row with the maximum "avg_netw_signal"
max_signal_index = netw_oper_signal['avg_netw_signal'].idxmax()

# Retrieve the corresponding row with the maximum "avg_netw_signal"
top_signal = netw_oper_signal.loc[max_signal_index]

# Display the row with the maximum "avg_netw_signal"
top_signal

Which network/operator has the highest average signal among the top 25% of recorded activities?

In [ ]:
# Calculate the threshold for the top 25% percentile of "record_count"
threshold = netw_oper_signal['record_count'].quantile(0.75)

# Filter the DataFrame to select rows with "record_count" greater than or equal to the threshold
top_25_percentile = netw_oper_signal[netw_oper_signal['record_count'] >= threshold]

# Find the row with the maximum "avg_netw_signal" within the filtered DataFrame
max_avg_netw_signal_row = top_25_percentile.loc[top_25_percentile['avg_netw_signal'].idxmax()]

# Display the row with the maximum "avg_netw_signal"
max_avg_netw_signal_row

Top Signal Strength by Operator and Net_Provider (top 10% record_count, excluding null values)

In [21]:
# SQL query: Top Signal Strength by Operator and Network (top 10% record_count, excluding null values)
query = f"""
WITH Top10Percent AS (
  
  # Subquery to prepare data to calculate the number of operators for each network
  SELECT
    net_provider,
    num_operator,
    record_count,
    RANK() OVER (ORDER BY record_count DESC) AS record_count_rank,
    avg_signal,
    quartile
  FROM (
    SELECT
      LOWER(net_provider) AS net_provider,
      
      # Calculate the number of distinct operators for each network
      COUNT(DISTINCT LOWER(operator)) AS num_operator,
      
      COUNT(*) AS record_count,
      ROUND(AVG(signal), 1) AS avg_signal,
      NTILE(10) OVER (ORDER BY COUNT(*) DESC) AS quartile
    FROM
      `{mobile_data_cleaned}`
    WHERE
    
    # Exclude rows with network 'null' (stored as string)
      net_provider != 'null'
    GROUP BY 1
  )
)

SELECT
  net_provider,
  num_operator,
  record_count,
  record_count_rank,
  avg_signal
FROM
  Top10Percent
WHERE
  quartile = 1
ORDER BY
  avg_signal DESC;
    """
# Execute the query
query_df(query)

,net_provider,num_operator,record_count,record_count_rank,avg_signal
0,vodafone,4,2648415,3,13.9
1,orange,12,2774131,2,13.2
2,yoigo,1,357852,4,13.0
3,movistar,5,4663075,1,12.9
4,jazztel,1,28351,6,11.9
5,eroski movil,1,39868,5,11.0


Top Network and Operator by Record Count for Each Province

In [ ]:
# SQL query
query = f"""
WITH ProvinceActivity AS (
  SELECT
    CASE
      WHEN LEFT(CAST(postal_code AS STRING), 2) = '08' THEN 'Barcelona'
      WHEN LEFT(CAST(postal_code AS STRING), 2) = '25' THEN 'Lleida'
      WHEN LEFT(CAST(postal_code AS STRING), 2) = '17' THEN 'Girona'
      WHEN LEFT(CAST(postal_code AS STRING), 2) = '43' THEN 'Tarragona'
      ELSE 'Not defined'
    END AS province,
    LOWER(network) AS network,
    LOWER(operator) AS operator,
    COUNT(*) AS record_count
  FROM `{mobile_data_cleaned}`
  GROUP BY province, network, operator
)

# Find the Rank of Network and Operator by Record Count for Each Province
, RankedNetwork AS (
  SELECT
    province,
    network,
    operator,
    record_count,
    RANK() OVER (PARTITION BY province ORDER BY record_count DESC) AS network_rank
  FROM ProvinceActivity
)

, RankedOperator AS (
  SELECT
    province,
    network,
    operator,
    record_count,
    RANK() OVER (PARTITION BY province ORDER BY record_count DESC) AS operator_rank
  FROM ProvinceActivity
)

# Select the Top Network and Top Operator for Each Province
SELECT
  p.province,
  n.network AS top_network,
  o.operator AS top_operator
  #n.record_count AS network_record_count,
  #o.record_count AS operator_record_count
FROM RankedNetwork n
JOIN RankedOperator o ON n.province = o.province AND n.network_rank = 1 AND o.operator_rank = 1
JOIN (SELECT DISTINCT province FROM ProvinceActivity) p ON n.province = p.province
ORDER BY p.province;
    """
# Execute the query
query_df(query)

### Description and activity

Description:
    
    - STATE_IN_SERVICE (0),
    - STATE_OUT_OF_SERVICE (1),
    - STATE_EMERGENCY_ONLY (2),
    - STATE_POWER_OFF (3)
    
User Activity:

    - IN_VEHICLE
    - STILL
    - ON_FOOT
    - TILTING
    - UNKNOWN
    - ON_BICYCLE

Activity Rank by Description with Overall Rank

In [ ]:
# SQL query
query = f"""
# Activity Rank by Description with Overall Rank
WITH ActivityRank AS (
  SELECT
    description,
    activity,
    RANK() OVER (PARTITION BY description ORDER BY COUNT(*) DESC) AS rank_by_description,
    COUNT(*) AS record_count
  FROM
    `{mobile_data_cleaned}`
  GROUP BY 1, 2
)

SELECT
  description,
  activity,
  rank_by_description,
  #record_count,
  RANK() OVER (ORDER BY rank_by_description, record_count DESC) AS overall_rank
FROM ActivityRank
ORDER BY 1, 4 ASC;
    """
# Execute the query
query_df(query)

Calculate activity rank by description and activity, including most frequent town and overall rank

In [ ]:
# SQL query
query = f"""
# Subquery to calculate the rank of activities by description
WITH ActivityRank AS (
  SELECT
    description,
    activity,
    RANK() OVER (PARTITION BY description ORDER BY COUNT(*) DESC) AS rank_by_description,
    COUNT(*) AS record_count
  FROM
    `bigquery-public-data.catalonian_mobile_coverage_eu.mobile_data_2015_2017`
  GROUP BY description, activity
),

# Subquery to find the most frequent town for each unique pair of description and activity
MostFrequentTown AS (
  SELECT
    description,
    activity,
    town_name,
    RANK() OVER (PARTITION BY description, activity ORDER BY COUNT(*) DESC) AS town_rank
  FROM
    `{mobile_data_cleaned}`
  GROUP BY description, activity, town_name
)

# Main query to combine the results and calculate overall rank
SELECT
  ar.description,
  ar.activity,
  ar.rank_by_description,
  RANK() OVER (ORDER BY ar.rank_by_description, ar.record_count DESC) AS overall_rank,
  mft.town_name AS most_frequent_town
FROM ActivityRank ar
# Join with the MostFrequentTown subquery to include the most frequent town
JOIN MostFrequentTown mft ON ar.description = mft.description AND ar.activity = mft.activity
# Filter for the most frequent town
WHERE mft.town_rank = 1 
ORDER BY 1, 3 ASC;
    """
# Execute the query
query_df(query)